In [7]:
pip install albumentations

     ---------------------------------------- 0.0/43.1 kB ? eta -:--:--
     ------------------ ------------------- 20.5/43.1 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 43.1/43.1 kB 529.5 kB/s eta 0:00:00
     ---------------------------------------- 0.0/66.6 kB ? eta -:--:--
     ------------------------------------ --- 61.4/66.6 kB 1.7 MB/s eta 0:00:01
     ---------------------------------------- 66.6/66.6 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/81.9 kB ? eta -:--:--
     ---------------------------------------- 81.9/81.9 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/67.4 kB ? eta -:--:--
     ---------------------------------------- 67.4/67.4 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/332.3 kB ? eta -:--:--
   ------------- -------------------------- 112.6/332.3 kB 3.3 MB/s eta 0:00:01
   ------------------------ --------------- 204.8/332.3 kB 2.5 MB/s eta 0:00:01
 

In [9]:
import numpy as np 
import os 
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image 
from torch.utils.data import Dataset,DataLoader 
from torchvision.utils import save_image 

In [11]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TRAIN_DIR = "data/train"
VAL_DIR = "data/val"
LEARNING_RATE = 2e-4
BATCH_SIZE = 16
NUM_WORKERS = 2
IMAGE_SIZE = 256
CHANNELS_IMG = 3
L1_LAMBDA = 100
LAMBDA_GP = 10
NUM_EPOCHS = 500
LOAD_MODEL = False
SAVE_MODEL = False
CHECKPOINT_DISC = "disc.pth.tar"
CHECKPOINT_GEN = "gen.pth.tar"

both_transform = A.Compose(
    [A.Resize(width=256, height=256),], additional_targets={"image0": "image"},
)

transform_only_input = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.ColorJitter(p=0.2),
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255.0,),
        ToTensorV2(),
    ]
)

transform_only_mask = A.Compose(
    [
        A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], max_pixel_value=255.0,),
        ToTensorV2(),
    ]
)

In [41]:
class MapDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.list_files = os.listdir(self.root_dir)

    def __len__(self):
        return len(self.list_files)

    def __getitem__(self, index):
        img_file = self.list_files[index]
        img_path = os.path.join(self.root_dir, img_file)
        image = np.array(Image.open(img_path))
        input_image = image[:, :600, :]
        target_image = image[:, 600:, :]

        augmentations = config.both_transform(image=input_image, image0=target_image)
        input_image = augmentations["image"]
        target_image = augmentations["image0"]

        input_image = config.transform_only_input(image=input_image)["image"]
        target_image = config.transform_only_mask(image=target_image)["image"]

        return input_image, target_image


if __name__ == "__main__":
    dataset = MapDataset("pix2pix-dataset")
    loader = DataLoader(dataset, batch_size=5)
    for x, y in loader:
        print(x.shape)
        save_image(x, "x.png")
        save_image(y, "y.png")
        import sys

        sys.exit()